In [1]:
#import libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchFrameException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.action_chains import ActionChains

import pymysql
import configparser
import mysql.connector

print('Libraries Imported.')

Libraries Imported.


In [2]:
# retruns a new web driver object
def createNewDriver():   

    options = webdriver.ChromeOptions() 
    # options.add_argument('--headless') # run driver without browser
    driver = webdriver.Chrome(executable_path='C:\Program Files\chromedriver\chromedriver.exe', 
                                options=options)

    return driver

In [3]:
# navigate driver to proper frame within Product Line tab
def navigateToProductList(driver):
    url = 'https://www.echo-usa.com/Support-Help/Parts-Lookup'

    driver.get(url)
    # navigate to product line tab
    frameset = driver.find_element_by_tag_name('frameset')

    frame = frameset.find_element_by_tag_name('frame')
    frame_name = frame.get_attribute('name')

    # grab last 6 chars of frame name since it changes everytime page is loaded
    frame_suffix = frame_name[11:17]
    # print(frame_suffix)

    # set driver in frame that contains necessary info
    driver.switch_to.frame(frame)

    # find product line tab
    product_line = driver.find_element_by_id("Tab2")

    # select product line tab
    product_line.click()
    # print(browser.page_source)

    # set names/IDs of frames based on 6 chars that were previously saved
    prodLineFrameID = 'iframeSearchProdLine_' + frame_suffix
    detailFrameID = 'plDetailFrame'
    dataFrameName = 'ARICAT6TREE_' + frame_suffix

    # navigate into frame that contains jstree of all products/parts
    frame = driver.find_element_by_id(prodLineFrameID)
    driver.switch_to.frame(frame)

    frame = driver.find_element_by_id(detailFrameID)
    driver.switch_to.frame(frame)

    frame = driver.find_element_by_name(dataFrameName)
    driver.switch_to.frame(frame)

    # click into ECHO jstree
    cur_node = driver.find_element_by_id('pn18')
    cur_node.click()

    return frame_suffix

In [4]:
def getNodes(unorderedList):
    # get all list items from unordered list param
    nodes = unorderedList.find_elements_by_tag_name("li")

    # iterate through all list items in current unordered list
    for node in nodes:
        # action = ActionChains(driver)
  
        # # perform the operation
        # action.move_to_element(node).perform()
        
        # grab and store node ID
        node_id = node.get_attribute('id')

        wait = WebDriverWait(driver, 10)
        node = wait.until(EC.presence_of_element_located((By.ID, node_id)))

        # grab and store ID of corresponding anchor
        anchor_id = node_id + '_anchor'

        # make driver wait for node to be clickable before clicking
        wait = WebDriverWait(driver, 10)
        # anchor = wait.until(EC.element_to_be_clickable((By.ID, anchor_id)))
        anchor = wait.until(EC.presence_of_element_located((By.ID, anchor_id)))
        anchor.click()
        
        text = anchor.text
        # print(text)
        
        if node.get_attribute('data-assembly-type') == "MD":
            serialNumIndex = text.find('S/N')
            print(serialNumIndex)
            if serialNumIndex != -1:
                model = text[0, (serialNumIndex - 1)]
                print("Model: " + model + '.')
                serialNum = text[serialNumIndex + 4]
                print("Serial Number: " + serialNum + '.')
                # sql = ("INSERT INTO tools (manufacturer, model, serial_number) VALUES (%s, %s)")
                # val = ("ECHO", )
                # myCursor.execute()
                print('Reached model file.')
            else:
                model = text
                print("Model: " + model + ".")

        # (base case) check if node is a parent, if false gather info from leaf node
        if node.get_attribute('data-assembly-is-parent') == "false" and node.get_attribute('data-assembly-thumb-url') is not None:
            print('Reached leaf node.')

            # if text == 'No Diagrams Required':
            #     print('skip')
            #     # TO DO: add parent data to sql table
            #     continue
            # else:
            #     getData()
        # else, click the element to expand child list
        else:
            # get xpath of current node
            xpath = '//li[@id = \'' + node_id + '\' and @aria-expanded = \'true\' and @aria-busy = \'false\']'

            # wait for list item to be visible
            wait = WebDriverWait(driver, 10)
            wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))

            # once visible get a new reference to it, as the old reference is most likely stale
            node = driver.find_element_by_xpath(xpath) 

            # find next unordered list element and call getParts recursively to iterate through next level
            getNodes(node.find_element_by_tag_name("ul"))

In [5]:
def getData():
    driver.switch_to.parent_frame()
    
    frame = driver.find_element_by_name('plDetail')
    # print(frame.get_attribute('name'))

    driver.switch_to.frame(frame)
    driver.switch_to.frame(2)

    table = driver.find_element_by_tag_name('table')
    # print(table.get_attribute('id'))

    for row in table.find_elements_by_tag_name('tr'):
        index = 0
        for cell in row.find_elements_by_tag_name('td'):
            if index > 2:
                print(cell.text)
            index += 1
    switchBack()
    # try:
    #     driver.switch_to.frame(2)

    #     table = driver.find_element_by_tag_name('table')
    #     # print(table.get_attribute('id'))

    #     for row in table.find_elements_by_tag_name('tr'):
    #         index = 0
    #         for cell in row.find_elements_by_tag_name('td'):
    #             if index > 2:
    #                 print(cell.text)
    #             index += 1
    #     switchBack()
    # except NoSuchElementException:
    #     switchBack()
    # except NoSuchFrameException:
    #     switchBack()

In [6]:
def switchBack():
    driver.switch_to.parent_frame()
    driver.switch_to.parent_frame()

    frameName = 'ARICAT6TREE_' + frame_suffix

    # navigate into frame that contains jstree of all products/parts
    frame = driver.find_element_by_name(frameName)
    driver.switch_to.frame(frame)


In [7]:
def connectMySQL():
    HOST = "ifixit-hiring.cxbmajqbo6pd.us-west-1.rds.amazonaws.com"
    USERNAME = "admin"
    PASSWORD = "mq76CKehrqTY7hgQZDRU"
    DATABASE = "tool_scraping"

    tool_db = mysql.connector.connect(
        host = HOST,
        user = USERNAME,
        password = PASSWORD,
        database = DATABASE
    )

    myCursor = tool_db.cursor()

    try:
        myCursor.execute("CREATE TABLE tools (id INT AUTO_INCREMENT PRIMARY KEY, manufacturer VARCHAR(255), model VARCHAR(255), serial_number VARCHAR(255))")
    except mysql.connector.ProgrammingError:
        return tool_db

    return tool_db

In [8]:
# create new webdriver to begin exploring target data
driver = createNewDriver()
frame_suffix = navigateToProductList(driver)

# tool_db = connectMySQL()
# myCursor = tool_db.cursor()

# select first ul that contains all subdirectories
productList = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//ul[contains(@class, \'jstree-container-ul\')][li[@aria-level = \'1\']]')))
# pass list of products into getNodes(), a recursive func
getNodes(productList)